In [1]:
import os

def check_subfolders_in_images(directory_path):
    """
    Checks the number of subfolders inside 'Images_001' for each main folder.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Number of subfolders inside 'Images_001' for each folder.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in folders:
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                print(f"Subfolders inside '{image_001_path}': {len(subfolders)}")
            else:
                print(f"No 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"Directory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
check_subfolders_in_images(directory_path)


Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image1a\Images_001': 671
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image2a\Images_001': 387
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image3a\Images_001': 352
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image4a\Images_001': 277
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image5a\Images_001': 22
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image6a\Images_001': 789


In [ ]:
import os
import numpy as np
from tqdm import tqdm

try:
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
except ImportError:
    print("TensorFlow is not installed. You can install it by running: pip install tensorflow")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    Stores all subfolders' images in a single array and prints its shape.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Shape of the array containing resized images for all subfolders.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                
                # Array to store all images for the current main folder
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    
                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
load_images_from_subfolders(directory_path)


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

# Custom PatchEncoder layer
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection = tf.keras.Sequential(
            [
                tf.keras.layers.Conv2D(2, (3, 3), strides=(1, 1), activation='relu'),
                tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
                tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
                tf.keras.layers.Dense(projection_dim),
            ]
        )
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Load model
model_path = r"E:\five_frame_model.h5"
custom_objects = {'PatchEncoder': PatchEncoder}

try:
    model = load_model(model_path, custom_objects=custom_objects)
    print("\nModel loaded successfully!")
except Exception as e:
    print(f"\nError loading model: {e}")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")

                # Make individual predictions for the current folder
                make_predictions(model, all_images_array, folder)
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

def make_predictions(model, data, folder_name):
    """
    Makes predictions on the given data and prints the count of each class for a specific folder.
    """
    print(f"\nMaking predictions for folder '{folder_name}'...")
    predictions = model.predict(data, batch_size=32, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    unique_classes, counts = np.unique(predicted_classes, return_counts=True)
    print(f"\nPredicted Class Distribution for '{folder_name}':")
    for cls, count in zip(unique_classes, counts):
        print(f"Class {cls}: {count}")

# Load images and make individual predictions
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
#load_images_from_subfolders(directory_path)



Model loaded successfully!


In [ ]:
# Load images and make individual predictions
directory_path = r"G:\datasets\2.Veliger_Data\May 27 2022  11-18\17 J H Park 11 Minutes"
load_images_from_subfolders(directory_path)


Processing main folders:   0%|                                                                   | 0/5 [00:00<?, ?it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   0%|           | 0/1058 [00:00<?, ?it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   0%|   | 1/1058 [00:00<02:52,  6.14it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   0%|   | 2/1058 [00:00<02:16,  7.73it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   0%|   | 4/1058 [00:00<01:56,  9.03it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   1%|   | 6/1058 [00:00<01:49,  9.57it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   1%|   | 8/1058 [00:00<01:38, 10.65it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':   1%|  | 10/1058 [00:00<01:27, 11.92it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  13%|▏| 134/1058 [00:09<01:00, 15.36it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  13%|▏| 136/1058 [00:09<00:58, 15.77it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  13%|▏| 139/1058 [00:09<00:51, 17.84it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  13%|▏| 141/1058 [00:09<00:50, 18.06it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  14%|▏| 143/1058 [00:09<00:52, 17.27it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  14%|▏| 145/1058 [00:10<00:54, 16.63it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  14%|▏| 147/1058 [00:10<00:55, 16.30it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  14%|▏| 149/1058 [00:10<01:01, 14.67it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  27%|▎| 281/1058 [00:18<00:47, 16.39it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  27%|▎| 283/1058 [00:18<00:50, 15.45it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  27%|▎| 285/1058 [00:18<00:54, 14.24it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  27%|▎| 287/1058 [00:18<00:56, 13.69it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  27%|▎| 289/1058 [00:18<00:51, 15.02it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  28%|▎| 291/1058 [00:19<00:47, 15.99it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  28%|▎| 293/1058 [00:19<00:47, 16.22it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  28%|▎| 295/1058 [00:19<00:48, 15.88it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  41%|▍| 435/1058 [00:27<00:34, 18.05it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  41%|▍| 437/1058 [00:27<00:33, 18.38it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  41%|▍| 439/1058 [00:27<00:36, 17.07it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  42%|▍| 441/1058 [00:27<00:37, 16.55it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  42%|▍| 444/1058 [00:28<00:38, 16.12it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  42%|▍| 446/1058 [00:28<00:40, 14.93it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  42%|▍| 449/1058 [00:28<00:36, 16.60it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  43%|▍| 452/1058 [00:28<00:36, 16.80it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  55%|▌| 580/1058 [00:37<00:34, 13.81it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  55%|▌| 582/1058 [00:37<00:32, 14.56it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  55%|▌| 584/1058 [00:37<00:30, 15.67it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  55%|▌| 586/1058 [00:37<00:32, 14.68it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  56%|▌| 588/1058 [00:37<00:30, 15.51it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  56%|▌| 590/1058 [00:38<00:31, 14.70it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  56%|▌| 592/1058 [00:38<00:33, 14.10it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  56%|▌| 594/1058 [00:38<00:33, 13.99it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  68%|▋| 720/1058 [00:46<00:21, 15.43it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  68%|▋| 722/1058 [00:46<00:22, 15.06it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  68%|▋| 724/1058 [00:47<00:23, 14.49it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  69%|▋| 726/1058 [00:47<00:25, 13.20it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  69%|▋| 728/1058 [00:47<00:23, 13.90it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  69%|▋| 730/1058 [00:47<00:22, 14.36it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  69%|▋| 732/1058 [00:47<00:23, 13.61it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  69%|▋| 734/1058 [00:47<00:22, 14.33it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  82%|▊| 865/1058 [00:56<00:09, 19.64it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  82%|▊| 868/1058 [00:56<00:10, 18.41it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  82%|▊| 870/1058 [00:56<00:11, 15.92it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  82%|▊| 872/1058 [00:56<00:13, 14.27it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  83%|▊| 874/1058 [00:56<00:12, 14.95it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  83%|▊| 877/1058 [00:57<00:10, 17.34it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  83%|▊| 879/1058 [00:57<00:10, 17.74it/s]
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  83%|▊| 883/1058 [00:57<00:08, 21.40it/s]
Processing subfolders in '17_27 J H Park

Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  95%|▉| 1007/1058 [01:06<00:04, 12.55it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  95%|▉| 1009/1058 [01:06<00:03, 12.51it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  96%|▉| 1011/1058 [01:06<00:03, 13.60it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  96%|▉| 1013/1058 [01:06<00:03, 13.47it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  96%|▉| 1015/1058 [01:06<00:03, 13.68it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  96%|▉| 1017/1058 [01:07<00:03, 12.48it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  96%|▉| 1019/1058 [01:07<00:02, 13.05it/s
Processing subfolders in '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a':  97%|▉| 1021/1058 [01:07<00:02, 14.06it/s
Processing subfolders in '17_27 J H Park


Subfolders inside 'G:\datasets\2.Veliger_Data\May 27 2022  11-18\17 J H Park 11 Minutes\17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a\Images_001': shape of images: (1058, 5, 28, 28, 3)

Making predictions for folder '17_27 J H Park 11 Minutes 300 Gal 12-3-21 Image1a'...
